In [1]:
import numpy as np
import pandas as pd

In [52]:
# возвращает упорядоченные нечетные чисела
def theta_N(N):
    # составляем последовательность чисел (a, b), где a - число, b = 0 без надчеркивания, b = 1 с надчеркиванием
    N_array = []
    i = N
    while i > 0:
        N_array.append([i, 0])
        if i % 2 == 1:
            i -= 1
        else:
            i //= 2

    # добавляем надчеркивания
    flag = True
    for i in range(1, len(N_array) - 1, 1):
        if N_array[i][0] % 2 == 0 and N_array[i - 1][0] % 2 == 1 and N_array[i + 1][0] % 2 == 1:
            N_array[i][1] = 1
        if flag:
            if N_array[i - 1][0] % 2 == 1 and N_array[i][0] % 2 == 0:
                N_array[i][1] = 1
                flag = False

    # построение последовательности нечетных чисел
    theta_old = []
    theta = [1]
    for i in range(len(N_array) - 2, -1, -1):
        theta_old = [j for j in theta]
        
        # алгоритмы перехода к следующей последовательности
        # от m к 2m без надчеркивания
        if N_array[i + 1][0] * 2 == N_array[i][0] and N_array[i][1] == 0:
            m = N_array[i + 1][0]
            theta = [None for j in range(2 * m)]
            for j in range(m):
                theta[2 * j] = theta_old[j]
                theta[2 * j + 1]     = 4 * m - theta_old[j]
        # от m к 2m с надчеркиванием
        elif N_array[i + 1][0] * 2 == N_array[i][0] and N_array[i][1] == 1:
            m = N_array[i + 1][0]
            theta = [None for j in range(2 * m)]
            for j in range(m):
                theta[2 * j] = theta_old[j]
                theta[2 * j + 1]     = 4 * m - theta_old[j] + 2
        # от 2m к 2m+1 без надчеркивания
        elif N_array[i + 1][0] + 1 == N_array[i][0] and N_array[i + 1][1] == 0:
            theta.append(2 * N_array[i + 1][0] + 1)
        # от 2m к 2m+1 с надчеркиванием
        elif N_array[i + 1][0] + 1 == N_array[i][0] and N_array[i + 1][1] == 1:
            theta.append(N_array[i + 1][0] + 1)
        else:
            print("Error theta_N")
    
    return theta

# возвращает индексы для tau в правильной последовательности
def tau_id(N):
    return [(i + 1) // 2 - 1 for i in theta_N(N)]

# возвращает шаг по времени
def tau_n(N, n, mu_max, mu_min):
    # в cos() (2n + 1), т.к. в программе n = 0, 1, ..., N - 1
    return 2 / (mu_max + mu_min + (mu_max - mu_min) * np.cos(np.pi * (2 * n + 1) / (2 * N)))

# максимальное mu
def mu_max_func(K, L, h_x, h_y):
    return 4 * ((np.cos(np.pi / (2 * K)) / h_x) ** 2 + (np.cos(np.pi / (2 * L)) / h_y) ** 2)

# минимальное mu
def mu_min_func(K, L, h_x, h_y):
    return 4 * ((np.sin(np.pi / (2 * K)) / h_x) ** 2 + (np.sin(np.pi / (2 * L)) / h_y) ** 2)

# правая часть
def f(x, y, t):
    return -25 * np.pi**2 * np.cos(3 * np.pi * x) * np.sin(4 * np.pi * y)

# начальное условие
def hi(x, y):
    return 1

# граничное условие
def phi(x, y, t):
    return 0

# аналитическое решение задачи
def u_analytical(x, y):
    return np.cos(3 * np.pi * x) * np.sin(4 * np.pi * y)


In [99]:
# сетка
K_0 = 6
koef_K = 8
K = koef_K * (K_0 - 1) + 1
x_grid, h_x = np.linspace(-1/2, 1/2, K, retstep=True)
x_grid_0 = np.linspace(-1/2, 1/2, K_0)  # сетка для печати

L_0 = 6
koef_L = 8
L = koef_L * (L_0 - 1) + 1
y_grid, h_y = np.linspace(0, 1, L, retstep=True)
y_grid_0 = np.linspace(0, 1, L_0)

# число итераций
N = 4024
# шаги по времени tau в правильной последовательности
mu_min = mu_min_func(K, L, h_x, h_y)
mu_max = mu_max_func(K, L, h_x, h_y)
tau_array = [tau_n(N, n, mu_max, mu_min) for n in tau_id(N)]

print(f"K = {K}, L = {L}, N = {N}")

# сеточная функция
u = np.full((K, L), np.nan)
# задание начального условия
for k in range(K):
    for l in range(L):
        u[k, l] = hi(x_grid[k], y_grid[l])
u_prev = []

# шаги по времени
t = 0  # время на текущем временном шаге
for count, tau in enumerate(tau_array):
    #print(f"count/N = {count}/{N}")

    t += tau
    u_prev = u.copy()
    u = np.full((K, L), np.nan)

    # задание граничных условий
    for k in range(K):
        u[k, 0]  = phi(x_grid[k], -1/2, t)
        u[k, -1] = phi(x_grid[k], 1/2, t)
    for l in range(L):
        u[0, l]  = phi(0, y_grid[l], t)
        u[-1, l] = phi(1, y_grid[l], t)
    
    # вычисление сеточной функции
    for k in range(1, K - 1, 1):
        for l in range(1, L - 1, 1):
            u[k, l] = u_prev[k, l] - tau * f(x_grid[k], y_grid[l], t) + \
                (tau / h_x ** 2) * (u_prev[k + 1, l] - 2 * u_prev[k, l] + u_prev[k - 1, l]) + \
                (tau / h_y ** 2) * (u_prev[k, l + 1] - 2 * u_prev[k, l] + u_prev[k, l - 1])

# численное решение для печати
u_print = u[::koef_K, ::koef_L]

u_an_print   = np.full((K_0, L_0), np.nan)  # аналитическое решение
u_both_print = [0 for k in range(K_0)]      # оба решения
for k in range(K_0):
    u_both_print[k] = [None for l in range(L_0)]
delta_abs = np.full((K_0, L_0), np.nan)  # абсолютная погрешность
delta_rel = np.full((K_0, L_0), np.nan)  # относительная погрешность
norma = 0  # норма относительной погрешности
for k in range(K_0):
    for l in range(L_0):
        u_an_print[k, l]    = u_analytical(x_grid_0[k], y_grid_0[l])
        u_both_print[k][l]  = str(round(u_print[k, l], 4)) + ' || ' + str(round(u_an_print[k, l], 4))
        delta_abs[k, l]     = abs(u_an_print[k, l] - u_print[k, l])
        if u_an_print[k, l] != 0:
            delta_rel[k, l] = abs(delta_abs[k, l] / u_an_print[k, l])
            norma = max(norma, delta_rel[k, l])
        else:
            delta_rel[k, l] = None

print(f"Норма: {norma}")

K = 41, L = 41, N = 4024
Норма: 1.0


In [100]:
df = pd.DataFrame({**{"Y/X": y_grid_0}, **{x_grid_0[i]: u_both_print[i] for i in range(K_0)}})
pd.options.display.float_format = '{:.4f}'.format

display(df)

,Y/X,-0.5,-0.3,-0.09999999999999998,0.10000000000000009,0.30000000000000004,0.5
0,0.0000,0.0 || -0.0,0.0 || -0.0,0.0 || 0.0,0.0 || 0.0,0.0 || -0.0,0.0 || -0.0
1,0.2000,0.0 || -0.0,-0.5629 || -0.559,0.3479 || 0.3455,0.3479 || 0.3455,-0.5629 || -0.559,0.0 || -0.0
2,0.4000,0.0 || 0.0,0.9108 || 0.9045,-0.5629 || -0.559,-0.5629 || -0.559,0.9108 || 0.9045,0.0 || 0.0
3,0.6000,0.0 || -0.0,-0.9108 || -0.9045,0.5629 || 0.559,0.5629 || 0.559,-0.9108 || -0.9045,0.0 || -0.0
4,0.8000,0.0 || 0.0,0.5629 || 0.559,-0.3479 || -0.3455,-0.3479 || -0.3455,0.5629 || 0.559,0.0 || 0.0
5,1.0000,0.0 || 0.0,0.0 || 0.0,0.0 || -0.0,0.0 || -0.0,0.0 || 0.0,0.0 || 0.0


In [101]:
pd.DataFrame({**{"Y/X": y_grid_0}, **{x_grid_0[i]:delta_abs[i] for i in range(K_0)}})

,Y/X,-0.5,-0.3,-0.09999999999999998,0.10000000000000009,0.30000000000000004,0.5
0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1,0.2000,0.0000,0.0039,0.0024,0.0024,0.0039,0.0000
2,0.4000,0.0000,0.0063,0.0039,0.0039,0.0063,0.0000
3,0.6000,0.0000,0.0063,0.0039,0.0039,0.0063,0.0000
4,0.8000,0.0000,0.0039,0.0024,0.0024,0.0039,0.0000
5,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000


In [72]:
print(f"Норма: {norma}")
print("Относительная погрешность")
pd.DataFrame({**{"Y/X": y_grid_0}, **{x_grid_0[i]:delta_rel[i] for i in range(K_0)}})

Норма: 2268836032748.5874
Относительная погрешность


,Y/X,-0.5,-0.4,-0.3,-0.19999999999999996,-0.09999999999999998,0.0,0.10000000000000009,0.20000000000000007,0.30000000000000004,0.4,0.5
0,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.1000,1.0000,0.1186,0.1186,0.1186,0.1185,0.1184,0.1185,0.1186,0.1186,0.1186,1.0000
2,0.2000,1.0000,0.1186,0.1188,0.1188,0.1182,0.1183,0.1182,0.1188,0.1188,0.1186,1.0000
3,0.3000,1.0000,0.1184,0.1184,0.1187,0.1186,0.1188,0.1186,0.1187,0.1184,0.1184,1.0000
4,0.4000,1.0000,0.1185,0.1184,0.1184,0.1187,0.1187,0.1187,0.1184,0.1184,0.1185,1.0000
5,0.5000,1.0000,297390843344.5592,606594706931.8827,2268836032743.2275,1059722079746.4773,905061281807.7963,1059722079746.3976,2268836032748.5874,606594706930.3677,297390843345.4855,1.0000
6,0.6000,1.0000,0.1185,0.1186,0.1186,0.1183,0.1183,0.1183,0.1186,0.1186,0.1185,1.0000
7,0.7000,1.0000,0.1186,0.1186,0.1183,0.1184,0.1182,0.1184,0.1183,0.1186,0.1186,1.0000
8,0.8000,1.0000,0.1184,0.1182,0.1182,0.1188,0.1187,0.1188,0.1182,0.1182,0.1184,1.0000
9,0.9000,1.0000,0.1184,0.1184,0.1184,0.1186,0.1186,0.1186,0.1184,0.1184,0.1184,1.0000


In [21]:
def tau_n_orig(N, n, mu_max, mu_min):
    return 2 / (mu_max + mu_min + (mu_max - mu_min) * np.cos(np.pi * (2 * n - 1) / (2 * N)))

# возвращает индексы для tau в правильной последовательности
def tau_id_orig(N):
    return [(i + 1) // 2 for i in theta_N(N)]

# сетка
K_0 = 6
koef_K = 1
K = koef_K * (K_0 - 1) + 1
x_grid, h_x = np.linspace(0, 1, K, retstep=True)
x_grid_0 = np.linspace(0, 1, K_0)  # сетка для печати

L_0 = 6
koef_L = 1
L = koef_L * (L_0 - 1) + 1
y_grid, h_y = np.linspace(0, 1, L, retstep=True)
y_grid_0 = np.linspace(0, 1, L_0)

# число итераций
N = 30
# шаги по времени tau в правильной последовательности
mu_min = mu_min_func(K, L, h_x, h_y)
mu_max = mu_max_func(K, L, h_x, h_y)
tau_array = [tau_n(N, n, mu_max, mu_min) for n in tau_id(N)]

theta_45 = theta_N(45)
theta_45_orig = [1,89, 43, 47, 21,69, 23,67,7,83,37,53,15, 75, 29, 61,3,87, 41, 49,19, 71, 25, 65,5,85,39,51,17, 73, 27, 63, 9,81,35,55,13, 77,31,59,11, 79, 33,57, 45]
tau_array_orig = [tau_n_orig(N, n, mu_max, mu_min) for n in tau_id_orig(N)]

print(f"theta_45_num  = {theta_45}")
print(f"theta_45_orig = {theta_45_orig}")
print(f"tau_num  = {tau_array}")
print(f"tau_orig = {tau_array_orig}")
maxim = 0
for t1, t2 in zip(tau_array_orig, tau_array):
    maxim = max(maxim, abs(t1 - t2))
print(maxim)

kurant_x = [t / h_x for t in tau_array]
print(f"max число Куранта: {max(kurant_x)}")
print(f"min число Куранта: {min(kurant_x)}")

theta_45_num  = [1, 89, 43, 47, 21, 69, 23, 67, 7, 83, 37, 53, 15, 75, 29, 61, 3, 87, 41, 49, 19, 71, 25, 65, 5, 85, 39, 51, 17, 73, 27, 63, 9, 81, 35, 55, 13, 77, 31, 59, 11, 79, 33, 57, 45]
theta_45_orig = [1, 89, 43, 47, 21, 69, 23, 67, 7, 83, 37, 53, 15, 75, 29, 61, 3, 87, 41, 49, 19, 71, 25, 65, 5, 85, 39, 51, 17, 73, 27, 63, 9, 81, 35, 55, 13, 77, 31, 59, 11, 79, 33, 57, 45]
tau_num  = [0.005362394519938927, 0.07398559063258778, 0.00956640949568534, 0.010474760864881201, 0.005977185757135178, 0.03058367210886563, 0.00647246071686374, 0.021978381639442737, 0.005445091872937524, 0.061168179021119, 0.008168974465390855, 0.012888987517265003, 0.005644505801969306, 0.043789410310367656, 0.007177888799141674, 0.016479006646185387, 0.005389780304180971, 0.0691386948797513, 0.008806877517324724, 0.011567061820672575, 0.0057927022957017646, 0.036537690928304875, 0.006794995870500683, 0.018927608642924997, 0.005529430942486063, 0.05222051167354667, 0.007631515345105977, 0.01450022960971682